In [591]:
import pandas as pd
import numpy as np
import random
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

data1 = pd.read_csv(r'C:\Users\Brahim\Desktop\prog\ger_data.csv')
labels = data1.iloc[:,-1]   # Enregistrer la varialbe target
data1 = data1.iloc[:,0:-1]; # Enleveer la colonne target
data1 = data1.drop(columns="Tel") # Enlever la colonne tel

In [592]:
def KMENAS(data):
    kmeans = KMeans(n_clusters=2)
    kmeans.fit(data);
    val = pd.crosstab(labels,kmeans.labels_).values
    """
    bcpg = val[0][1-val[0].argmax()]/(val[0][1-val[0].argmax()]+val[0][val[0].argmax()])
    gcpb = val[1][val[0].argmax()]/(val[1][1-val[0].argmax()]+val[1][val[0].argmax()])   
    """
    x1,x2,x3,x4 = val[0][0],val[0][1],val[1][0],val[1][1]
    cl_0 = [x1,x3]
    cl_1 = [x2,x4]
    cl = [cl_0,cl_1]
    if x1+x3 < x2+x4 :
        bcpg = x1/(x1+x2)
        gcpb = x4/(x4+x3)
    else:
        bcpg = x3/(x3+x4)
        gcpb = x2/(x2+x1)
    print("Bad Clients predicted Good : {1:2.2f}% \nGood Clients predicted Bad : {0:2.2f}% ".format(bcpg,gcpb))
    return pd.crosstab(labels,kmeans.labels_)

# Initialising Data

In [593]:
data_cat = data1.select_dtypes(include=['object'])
data_cat_og = data1.select_dtypes(include=['object'])
data_num_og = data1._get_numeric_data()
data_num = data1._get_numeric_data()

for col in data_cat:
    onehot = pd.get_dummies(data_cat[col])
    data_cat = data_cat.drop(col,axis = 1)
    data_cat = data_cat.join(onehot)
    
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_num_sc = pd.DataFrame(scaler.fit_transform(data_num), columns=data_num.columns)

data = [data_num_sc,data_cat]
data_join = pd.concat(data,axis=1)

In [594]:
datalist = ["data_num_sc","data_cat","data_join"]

In [595]:
KMENAS(data_join)

Bad Clients predicted Good : 0.49% 
Good Clients predicted Bad : 0.42% 


col_0,0,1
Actual_Score,,
1,294,406
2,153,147


#  First Models

### Quanti

In [585]:
KMENAS(data_num_sc)

Bad Clients predicted Good : 0.16% 
Good Clients predicted Bad : 0.85% 


col_0,0,1
Actual_Score,,
1,591,109
2,254,46


In [586]:
reduced_data = PCA(n_components=2).fit_transform(data_num_sc)
reduced_num = pd.DataFrame(reduced_data)
KMENAS(reduced_num)

Bad Clients predicted Good : 0.57% 
Good Clients predicted Bad : 0.47% 


col_0,0,1
Actual_Score,,
1,328,372
2,128,172


### Quali

In [587]:
KMENAS(data_cat)

Bad Clients predicted Good : 0.49% 
Good Clients predicted Bad : 0.42% 


col_0,0,1
Actual_Score,,
1,294,406
2,153,147


In [588]:
reduced_data = PCA(n_components=2).fit_transform(data_cat)
reduced_cat = pd.DataFrame(reduced_data)
KMENAS(reduced_cat)

Bad Clients predicted Good : 0.43% 
Good Clients predicted Bad : 0.45% 


col_0,0,1
Actual_Score,,
1,317,383
2,172,128


### Both

In [589]:
KMENAS(data_join)

Bad Clients predicted Good : 0.42% 
Good Clients predicted Bad : 0.49% 


col_0,0,1
Actual_Score,,
1,406,294
2,147,153


In [590]:
reduced_data = PCA(n_components=2).fit_transform(data_join)
reduced_join = pd.DataFrame(reduced_data)
KMENAS(reduced_join)

Bad Clients predicted Good : 0.43% 
Good Clients predicted Bad : 0.45% 


col_0,0,1
Actual_Score,,
1,316,384
2,170,130
